Setting up

In [3]:
import sklearn
import pandas as pd
import numpy as np

input_prefix = "../input/anz_crash_20200903_fix/"
cas_fp = input_prefix + "Casualties.csv"
crash_fp = input_prefix + "Crash2.csv"
dt_fp = input_prefix + "DateTime.csv"
desc_fp = input_prefix + "Description.csv"
loc_fp = input_prefix + "Location2.csv"
veh_fp = input_prefix + "Vehicles.csv"

Crash.csv is the "central" csv file that the other csv files connect to.

Rows in crash.csv contain id values that link to description, datetime, location, casualties, and vehicles.

The dataset source is https://www.kaggle.com/datasets/mgray39/australia-new-zealand-road-crash-dataset?resource=download-directory&select=anz_crash_20200903_fix and the link contains a graphic which clearly explains how all the files link together.

In order to make this data usable, all the data needs to be combined using the id information.

Combine all csv files together

In [13]:
cas_dat = pd.read_csv(cas_fp)
crash_dat = pd.read_csv(crash_fp)
dt_dat = pd.read_csv(dt_fp)
desc_dat = pd.read_csv(desc_fp)
loc_dat = pd.read_csv(loc_fp)
veh_dat = pd.read_csv(veh_fp)

# create new dataframe with features from datasets and crash data

data = pd.DataFrame(cas_dat[:] + dt_dat[:] + desc_dat[:] + loc_dat[:] + veh_dat[:])
data = data[0:0]
data = pd.DataFrame(crash_dat + data)

# cross reference the feature data in crash with other datasets to build the new dataframe
print(data)

C:\Users\thr0w\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\thr0w\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,6,7,8,9,10,11,12,13,14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\thr0w\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


         DCA_code  Unnamed: 0  animals  approximate  bicycle  bus  car_4x4  \
0             NaN         NaN      NaN          NaN      NaN  NaN      NaN   
1             NaN         NaN      NaN          NaN      NaN  NaN      NaN   
2             NaN         NaN      NaN          NaN      NaN  NaN      NaN   
3             NaN         NaN      NaN          NaN      NaN  NaN      NaN   
4             NaN         NaN      NaN          NaN      NaN  NaN      NaN   
...           ...         ...      ...          ...      ...  ...      ...   
1519450       NaN         NaN      NaN          NaN      NaN  NaN      NaN   
1519451       NaN         NaN      NaN          NaN      NaN  NaN      NaN   
1519452       NaN         NaN      NaN          NaN      NaN  NaN      NaN   
1519453       NaN         NaN      NaN          NaN      NaN  NaN      NaN   
1519454       NaN         NaN      NaN          NaN      NaN  NaN      NaN   

         car_sedan  car_station_wagon  car_utility  ...  taxi  